In [1]:
from utils import *
import geopandas as gpd
import pandas as pd
import sumolib
from skmob.tessellation import tilers
import json


import warnings
warnings.filterwarnings("ignore")

# Create the OD Matrix

#### File paths

In [2]:
#trajectories
mobility_data_path = "./data/datasets/trajset_D_1h.csv"

#road network
road_network_path = "../sumo_simulation_data/milan_medium_v5.net.xml"

#shapefile
shapefile_path = "./data/milan_medium.geojson"

#outputs
od_matrix_path = "./data"
dict_tile_edges_path =  "./data"

#### Load the real mobility data

In [3]:
traj_D = pd.read_csv(mobility_data_path)
traj_D = skmob.TrajDataFrame(traj_D[['uid', 'datetime', 'lat', "lng"]], latitude='lat', longitude='lng', 
                                             user_id='uid', datetime='datetime')

#### Load the shapefile of the geographic area of interest

In [4]:
shape = gpd.GeoDataFrame.from_file(shapefile_path)

#### Create a squared tessellation of the city (size of 1km)

In [5]:
tile_size_meters = 1000
tessellation = tilers.tiler.get('squared', base_shape=shape, meters=tile_size_meters)

#### Compute the OD matrix

In [6]:
od_matrix = compute_od_matrix(traj_D, tessellation, traj_id="uid", self_loops=True)

#### Save the OD matrix to compute the Mobility Demand

In [7]:
with open(od_matrix_path+'/od_matrix.npy', 'wb') as f:
    np.save(f, od_matrix)

#### Load the road network

In [8]:
road_network = sumolib.net.readNet(road_network_path, withInternal=False)

#### Assign road network edges to the corresponding tile

In [9]:
dict_tile_edges = create_dict_tile_edges(road_network, tessellation, exclude_roundabouts=True)

In [10]:
output_file = open(dict_tile_edges_path+"/dict_tile_edges.json", "w")
json.dump(dict_tile_edges, output_file)
output_file.close()